In [ ]:
!wget https://raw.githubusercontent.com/alimbekovKZ/ods_homework/master/data/wiki_machine_learning.csv

--2020-11-09 12:06:07--  https://raw.githubusercontent.com/alimbekovKZ/ods_homework/master/data/wiki_machine_learning.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31043 (30K) [text/plain]
Saving to: ‘wiki_machine_learning.csv.3’

wiki_machine_learni 100%[===================>]  30.32K  --.-KB/s    in 0.01s   

2020-11-09 12:06:07 (2.13 MB/s) - ‘wiki_machine_learning.csv.3’ saved [31043/31043]



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping

In [ ]:
df = pd.read_csv('wiki_machine_learning.csv', sep = ' ')
df = df[df['count'] != 0]
df.head()

,date,count,lang,page,rank,month,title
81,2015-01-01,1414,en,Machine_learning,8708,201501,Machine_learning
80,2015-01-02,1920,en,Machine_learning,8708,201501,Machine_learning
79,2015-01-03,1338,en,Machine_learning,8708,201501,Machine_learning
78,2015-01-04,1404,en,Machine_learning,8708,201501,Machine_learning
77,2015-01-05,2264,en,Machine_learning,8708,201501,Machine_learning


In [ ]:
df.date = pd.to_datetime(df.date)

In [ ]:
predictions = 30

df = df[['date', 'count']]
df.columns = ['ds', 'y']
train_df = df[:-predictions].copy()

test_data = df[-predictions:].copy()

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler = scaler.fit(np.expand_dims(train_df.y, axis=1))

In [ ]:
train_data = scaler.transform(np.expand_dims(train_df.y, axis=1))

test_data = scaler.transform(np.expand_dims(test_data.y, axis=1))

In [ ]:
data_x = np.array(train_data).reshape(-1,1)

In [ ]:
train_x = np.reshape( data_x, (data_x.shape[0], 1, data_x.shape[1]))

In [ ]:
train_x.shape

(353, 1, 1)

In [ ]:
test_x = np.array(test_data).reshape(-1,1)

In [ ]:
test_x = np.reshape(test_x, (test_x.shape[0], 1, test_x.shape[1]))

In [ ]:
test_x.shape

(30, 1, 1)

In [ ]:
train_y = df[:-predictions].y.values

In [ ]:
test_y = df[353:].y.values

In [ ]:
stop = EarlyStopping(monitor='loss',min_delta=0.000000000001, patience=15)

model = Sequential()
model.add(LSTM(128, input_shape=(train_x.shape[1], train_x.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=['mape'])

history = model.fit(
    train_x, 
    train_y, 
    epochs=10000, 
    batch_size=100, 
    validation_data=(test_x, test_y), 
    verbose=1, 
    shuffle=False,
    callbacks=[stop]
)

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
preds = model.predict(test_x)
preds = scaler.inverse_transform(preds)

test = scaler.inverse_transform(test_y.reshape(-1, 1))
#test = test_y.reshape(-1, 1)
print(mean_absolute_percentage_error(test, preds))
print(mean_absolute_percentage_error(sum(test), sum(preds)))